# Implementing Trading with Machine Learning Regression - Part 4

In this notebook, we will use signals and compute trade details and strategy analytics

The key steps for this notebook are:
1. Import the data
2. Analyze the Performance
3. Trade Details and Strategy Analytics
4. Visualise the Entry and Exit
5. Strategy Analysis

In [1]:
## Import the libraries
import numpy as np 
import pandas as pd 

# For Plotting
import matplotlib.pyplot as plt 
%matplotlib inline
plt.style.use('seaborn-darkgrid')

# To ignore unwanted warnings
import warnings
warnings.filterwarnings("ignore")

### Import Data
We will import the file `regression_strategy_data.csv`. This file has predicted  high, low, signals, gold ETF returns and strategy returns.

In [2]:
strategy_data = pd.read_csv('data/strategy_data.csv', parse_dates=['Date'], index_col='Date')
strategy_data.head()

,Close,High,P_H,Low,P_L,Signal,gld_returns,strategy_returns
Date,,,,,,,,
2017-08-10,122.209999,122.440002,122.581708,122.029999,121.514771,0,NaN,NaN
2017-08-11,122.790001,122.830002,122.747055,122.029999,121.670863,-1,0.004746,0.000000
2017-08-14,121.919998,122.239998,122.504406,121.680000,121.439645,0,-0.007085,0.007085
2017-08-15,120.980003,121.230003,121.153842,120.580002,120.091055,-1,-0.007710,-0.000000
2017-08-16,121.870003,122.059998,121.240462,120.660004,120.164585,-1,0.007357,-0.007357


### Analyze the performannce

First, we will use the Sharpe ratio to analyze the performance. Later in this notebook, we will examine our strategy in detail.

#### Sharpe Ratio
Sharpe ratio is a measure for calculating risk-adjusted return. It is the ratio of the excess return (over risk-free rate) per unit of volatility or standard deviation. The formula for Sharpe Ratio is given by,

`Sharpe ratio = (sqrt(N) x mean(strategy returns - Rf)) / (std dev (Strategy returns - Rf))`

where:
- N is the number of trading days in a year
- Rf is the risk-free rate
- Std dev is standard deviation

In [5]:
# The below function returns the Sharpe ratio for the excess returns
def annualized_sharpe_ratio(returns, N=252):
    return np.sqrt(N) * returns.mean() / returns.std()

# Calculate the excess daily return assuming an annual risk free rate of return of 5%
excess_daily_strategy_return = strategy_data['strategy_returns'] - 0.05/252

# Calculate the annualized sharpe ratio
sharpe = annualized_sharpe_ratio(excess_daily_strategy_return)
print('Sharpe - ', sharpe)

Sharpe -  0.1951213438453711
